# Qmod Tutorial - Part 1

In this tutorial, we will learn the basics of modeling quantum algorithms using the Qmod language and its accompanied function library.\
We will learn to use functions, operators, quantum variables, and quantum types. 

Let's begin with a simple code example:

In [11]:
from classiq import *

@qfunc
def foo(q: QBit) -> None:
    X(q)
    H(q)

@qfunc
def main(q: Output[QBit]) -> None:
    allocate(1,q)
    foo(q)

qmod  = create_model(main)

The function `foo` is defined to get an input variable `q` of type `QBit` and apply on it `X` gate, followed by `H` gate.\
The function `main` is defined with a single `Output` variable `q` of type `QBit`. It first allocates a qubit to `q`, than calls `foo` to act on it.
A Quantum model `qmod` is created based on the function `main`, so that it can later be synthesized and executed.

What results do we expect when executing this model?
- By calling `allocate`, `q` is initialized in the default state $|0\rangle$.
- Then, foo is called:
  - It applies `X` (NOT gate), changing `q`'s state to $|1\rangle$.
  - Then it applies `H` (Hadamard gate), resulting in the superposition $\frac{1}{\sqrt{2}} (|0\rangle - |1\rangle)$.

When executing the model, the output variable `q` is sampled. We can run the following code and make sure that the states $|0\rangle$ and $|1\rangle$ are sampled roughly equally:

In [12]:
job = execute(synthesize(qmod))
job.get_sample_result().parsed_counts

[{'q': 1}: 1032, {'q': 0}: 1016]

## Qmod Fundementals
This simple model demonstrates several features that are essential in every Qmod code:

1. The `@qfunc` decorator:\
   Qmod is a quantum computing language embeded into Python. The decorator `@qfunc` marks a Qmod function, so that the Qmod interpreter is called to handle it. The decorator is used in every quantum function definition.

2. The function `main`:\
   Any Qmod code that is intended to be synthesized into an executable quantum program must define a function `main`. The function `main` acts as the program's quantum entry point - it specifies the inputs and outputs of the quantum program, that is, its interface with the external classical execution logic. Other functions (`foo` as an example) can of course be defined and called, but if we track back the function calls they always point back to `main`.

3. Quantum variables initialization:\
   Every quantum variable must be initialized. Initialization means __allocating qubits to the variable and determining its initial state__. There are several ways to initialize a quantum variable:
   - The most basic way demonstrated above is calling `allocate`. This method requires specifying directly the number of qubits to be allocated to the variable, and the state of all these qubits is automatically set to the default $|0\rangle$. 
   - More advanced methods (to be covered later) include assigning an arithmetic expresssion to the variable, calling state-preparation function and more - such methods automatically determine the number of qubits to allocate based on the information that we wish the variable to represent.
   
4. The `Output` modifier:\
   The `Output` modifier indicates that a quantum variable is not initialized outside the scope of the function, and hence must be initialized inside the scope of the function.\
   For example, in the above `main` function definition, `q` is declared with the `Output` modifier, so `main` expects it to not yet be initialized when called. `foo`, on the other hand, declares its input `q` without the modifier, hence it is essential to initialize `q` before passing it to `foo` (see the call to `allocate` in the code).
   - Since `main` is the quantum entry point, all of its inputs must be declared with the `Output` modifier. Please take a moment to think about it: suppose that we would decalre an input `x` to `main` without the `Output` modifier. It follows that `x` must have been initialized before calling `main`, and was passed to main already initialized, but it would be impossible because `main` is the origin of any quantum logic - all its external interfaces are classical.

### Exercise #0
Rewrite the above model, so that `q` is initilized inside `foo`.
Hint: 